# Fast Prompting en Acción: Desentrañando la Magia
## Proyecto Final - Alex Quispe

### 📌 1. Introducción

En este proyecto se presenta una prueba de concepto (POC) que utiliza técnicas de **Fast Prompting** para resolver el problema de diseñar planes de ejercicio y alimentación personalizados.

**Problema:** Muchas personas buscan mejorar su salud pero enfrentan planes genéricos que no consideran sus limitaciones, tiempo ni preferencias.

**Solución:** Un asistente automatizado que genere rutinas de entrenamiento y alimentación personalizadas, además de material visual motivador.

### 📌 2. Configuración inicial

**Importante:** Antes de ejecutar este notebook, configura tu API Key de OpenAI como variable de entorno:

```bash
export OPENAI_API_KEY="tu_api_key_aqui"
```

O puedes configurarla directamente en el código (no recomendado para producción).

In [ ]:
# Instalación de dependencias (ejecutar solo si es necesario)
# !pip install openai

from openai import OpenAI
import os

# Configura tu API Key
client = OpenAI()
# Si no tienes la variable de entorno, descomenta y usa la siguiente línea:
# client = OpenAI(api_key="tu_api_key_aqui")

print("✅ Configuración completada")

### 📌 3. Prompts base para Fast Prompting

Definimos plantillas de prompts que usaremos en Fast Prompting. Se reemplazan `{campos}` con los datos del usuario.

In [ ]:
prompt_plan = """
Eres un entrenador personal virtual con conocimientos básicos de nutrición.
Recibirás los datos del usuario y debes generar:

1) Plan de entrenamiento semanal detallado.
2) Plan de comidas simple para 7 días.
3) Lista de compras agrupada por categorías.
4) Consejos de progresión y seguridad.

Datos del usuario:
Edad: {edad}
Sexo: {sexo}
Peso: {peso} kg
Altura: {altura} cm
Objetivo: {objetivo}
Nivel: {nivel}
Lesiones/limitaciones: {lesiones}
Disponibilidad: {dias_por_semana} días, {minutos_por_sesion} min/sesión
Equipo disponible: {equipo}
Preferencias alimentarias: {preferencias}

Entrega la respuesta estructurada en secciones numeradas.
"""

prompt_adaptacion = """
Toma el plan anterior y genera una versión alternativa que evite
impacto en {zona_lesionada}. Mantén la misma duración y objetivo,
proponiendo ejercicios sustitutos y modificaciones de intensidad.
"""

prompt_motivacion = """
Genera 7 mensajes motivacionales cortos, uno por día, orientados al
objetivo {objetivo}. Cada mensaje debe incluir un consejo práctico
y una micro-tarea sencilla.
"""

print("✅ Prompts definidos")

### 📌 4. Funciones auxiliares

In [ ]:
def generar_respuesta(prompt: str) -> str:
    """Llama a la API con un prompt y devuelve la respuesta."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=800
    )
    return response.choices[0].message.content

print("✅ Funciones auxiliares definidas")

### 📌 5. Caso de prueba con datos ficticios

Definimos un usuario de ejemplo para probar nuestro sistema de Fast Prompting.

In [ ]:
usuario = {
    "edad": 30,
    "sexo": "F",
    "peso": 72,
    "altura": 168,
    "objetivo": "pérdida de peso",
    "nivel": "principiante",
    "lesiones": "rodilla derecha (leve, sin cirugía)",
    "dias_por_semana": 4,
    "minutos_por_sesion": 45,
    "equipo": "banda elástica, mancuernas ligeras, silla",
    "preferencias": "vegetariana"
}

print("👤 Datos del usuario configurados:")
for key, value in usuario.items():
    print(f"  {key}: {value}")

### 📌 6. Generación del plan inicial

In [ ]:
# Generamos el plan inicial
prompt = prompt_plan.format(**usuario)
plan_inicial = generar_respuesta(prompt)

print("🏋️‍♀️ PLAN INICIAL GENERADO:")
print("=" * 50)
print(plan_inicial)

### 📌 7. Adaptación del plan con Fast Prompting

Ahora adaptamos el plan para tener en cuenta la lesión en la rodilla.

In [ ]:
prompt = prompt_adaptacion.format(zona_lesionada="rodilla derecha")
plan_adaptado = generar_respuesta(prompt)

print("🦵 PLAN ADAPTADO PARA LESIÓN:")
print("=" * 50)
print(plan_adaptado)

### 📌 8. Mensajes motivacionales

In [ ]:
prompt = prompt_motivacion.format(objetivo=usuario["objetivo"])
mensajes = generar_respuesta(prompt)

print("💪 MENSAJES MOTIVACIONALES:")
print("=" * 50)
print(mensajes)

### 📌 9. Experimentación con Fast Prompting

Ejemplo: pedimos al modelo que genere el plan en formato tabla en lugar de texto libre.

In [ ]:
prompt_tabla = prompt_plan.format(**usuario) + """
IMPORTANTE: Entrega los planes en formato de tabla Markdown para facilitar la lectura.
"""

plan_tabla = generar_respuesta(prompt_tabla)

print("📊 PLAN EN FORMATO TABLA:")
print("=" * 50)
print(plan_tabla)

### 📌 10. Función para probar con nuevos usuarios

Creamos una función reutilizable para generar planes para cualquier usuario.

In [ ]:
def generar_plan_completo(datos_usuario, formato_tabla=False):
    """Genera un plan completo para un usuario."""
    
    # Elegir el prompt base
    prompt_base = prompt_plan
    if formato_tabla:
        prompt_base += "\nIMPORTANTE: Entrega los planes en formato de tabla Markdown."
    
    # Generar plan principal
    prompt = prompt_base.format(**datos_usuario)
    plan = generar_respuesta(prompt)
    
    # Generar mensajes motivacionales
    prompt_mot = prompt_motivacion.format(objetivo=datos_usuario["objetivo"])
    motivacion = generar_respuesta(prompt_mot)
    
    return {
        "plan_principal": plan,
        "motivacion": motivacion
    }

# Ejemplo de uso con nuevo usuario
nuevo_usuario = {
    "edad": 25,
    "sexo": "M",
    "peso": 80,
    "altura": 175,
    "objetivo": "ganancia de masa muscular",
    "nivel": "intermedio",
    "lesiones": "ninguna",
    "dias_por_semana": 5,
    "minutos_por_sesion": 60,
    "equipo": "gimnasio completo",
    "preferencias": "omnívoro"
}

print("🆕 PROBANDO CON NUEVO USUARIO:")
resultado = generar_plan_completo(nuevo_usuario, formato_tabla=True)
print("\n📋 PLAN:")
print(resultado["plan_principal"])
print("\n💪 MOTIVACIÓN:")
print(resultado["motivacion"])

### 📌 11. Conclusiones

**Logros del proyecto:**
- ✅ Usando Fast Prompting pudimos optimizar la salida (tablas, secciones claras)
- ✅ Se redujo el número de consultas al agrupar varias instrucciones en un solo prompt
- ✅ El sistema permite personalización rápida para distintos usuarios
- ✅ Implementación modular y reutilizable

**Limitaciones identificadas:**
- 💰 Costo de API por consulta
- 🖼️ Calidad de imágenes (requiere integración externa)
- 📏 Límite de tokens por respuesta

---

**¡Notebook completado!** 🎉

Para usar este notebook:
1. Configura tu API Key de OpenAI
2. Ejecuta todas las celdas en orden
3. Modifica los datos de usuario según tus necesidades
4. Experimenta con diferentes prompts y configuraciones